### 大作业：爬取腾讯新闻

1. 了解ajax加载
2. 通过chrome的开发者工具，监控网络请求，并分析
3. 用selenium完成爬虫
4. 具体流程如下：
<br>用selenium爬取https://news.qq.com/ 的热点精选
![1585810800%281%29.png](1585810800%281%29.png)
热点精选至少爬50个出来，存储成csv
每一行如下
标号（从1开始）,标题,链接,...（前三个为必做，后面内容可以自己加）
![1585810759%281%29.png](1585810759%281%29.png)


In [1]:
import time, requests, json, re, random
from selenium import webdriver

from lxml import etree

#### 1、打开腾讯新闻网页

In [4]:
driver = webdriver.Chrome()
driver.get('https://news.qq.com/')
# 将窗口调整最大
driver.maximize_window()
# 休息5s
time.sleep(5)
#此处需添加页面滚动加载功能

### 2、【热点新闻】默认加载10条，随着页面滚动向下，动态加载增加
- 调用‘window.scrollBy’执行页面滚动
- 利用random对单次滚动距离、滚动间隔时间进行选取，模拟人工操作

In [5]:
import random
for i in range(20):
    pixel = random.randint(800,1000)
    driver.execute_script(f'window.scrollBy(0,{pixel})')
    time.sleep(random.random()+1)


### 3、获取网页html【热点新闻】部分

In [6]:
driver.find_element_by_xpath('//ul[@class="list"]')

<selenium.webdriver.remote.webelement.WebElement (session="ec21e417877c63e16631dd23c4b68f38", element="77cd9798-2ef1-4bc5-a50d-d11b8666fd0c")>

### 4、读取新闻标题、链接、发布媒体等信息

In [8]:
#实现对当前页面中已加载热点新闻列表的爬取
tree = etree.HTML(driver.page_source) 
newstree = tree.xpath('//*[@id="List"]/div/ul[2]')[0].xpath('li')
count = 0
newslist=list()
for news in newstree:
    source = 'aaa'
    if news.xpath('div/h3/a/text()') != []:
        title = news.xpath('div/h3/a/text()')[0]
        nurl  = news.xpath('div/h3/a/@href')[0]
        source= news.xpath('div/div[@class="binfo cf"]/div/a/text()')[0]
    else:
        title = news.xpath('h3/a/text()')[0]
        nurl  = news.xpath('h3/a/@href')[0]
        source= news.xpath('div[@class="binfo cf"]/div/a/text()')[0]
    count += 1
    print(f'{count:<2d}, {title} , {nurl} , {source}')
    newslist.append([count,title,nurl,source])


1 , 当当电话会议实录：李国庆抢公章行为非法 俞渝持股52.23% , https://new.qq.com/zt/template/?id=TEC2020042600521000 , 腾讯新闻
2 , 哈尔滨“1传80＋”，副市长陈远飞再被中央纪委通报 , https://new.qq.com/omn/20200426/20200426A0KWHQ00.html , 长安街知事
3 , 市委书记、市长请了10位企业家一起吃饭，现场画面曝光 , https://new.qq.com/omn/20200426/20200426A0GYH700.html , 北京青年报政知道
4 , 河北保定一家4口被害案警方再发悬赏通告 更多案件细节披露 , https://new.qq.com/omn/20200426/20200426A0I5IH00.html , 北青Qnews
5 , 31省份最低工资调整出炉：上海2480元领跑 北京时薪最高 , https://new.qq.com/omn/20200426/20200426A0G0WS00.html , 中国网
6 , 媒体：“多国女子想嫁到中国”，鸡血含量高达100％ , https://new.qq.com/omn/20200426/20200426A0DVC700.html , 沸腾
7 , 非洲人在华遭虐待？法新社核查网络视频揭开真相 , https://new.qq.com/omn/20200426/20200426A0ALM200.html , 中国新闻网
8 , 河北广宗自来水灼伤村民 记者探访涉事企业：大门紧闭已停工 , https://new.qq.com/omn/20200426/20200426V0HHDQ00.html , 新京报我们视频
9 , 探访当当网总部：当当网报警称李国庆闯入抢走公章 , https://new.qq.com/omn/20200426/20200426V0JNY500.html , 老板联播
10, 现场画面！武汉最后一批在院新冠肺炎患者出院 , https://new.qq.com/omn/20200426/20200426V0HGUW00.html , 长江直播
11, 投资人：中行表示暂不追缴原油宝保证金欠款 也暂不纳入征信 , https://new.qq.com

In [9]:
driver.quit()

In [10]:
#最终结果，存入csv中
import pandas as pd
name=['number','title','url','source']
df=pd.DataFrame(columns=name,data=newslist)
df.to_csv('tencent_news.csv',index=False)